In [1]:
import pandas as pd
import datetime
import urllib
import numpy as np
from io import BytesIO
from io import StringIO

In [2]:
def get_data(symbol,start,end):
    base_url = 'https://query1.finance.yahoo.com/v7/finance/download/'
    #EJ Symbol: ^GSPC
    base_url = base_url+symbol+'?%s'
    payload = urllib.parse.urlencode({'period1': start, 'period2': end, 'interval': '1d', 'events':'history'})

    req = urllib.request.urlopen(base_url % payload)
    req_read = req.read()
    file_download = BytesIO(req_read)

    df = pd.read_csv(file_download)

    df['Close'].replace('', np.nan, inplace=True)
    df.dropna(subset=['Close'], inplace=True)
    
    df['Symbol'] = symbol
    df['actualizacion'] = datetime.datetime.now()

    return df

In [3]:
def upload_azure():
    #Verificar si la ruta existe.
    #symbols = ['^GSPC','GC=F','BG','SI=F','^DJI','GLEN.L','ADM']

    data = pd.DataFrame()


    symbol = 'USDCAD=X'
    max_date = datetime.datetime(2010, 1, 4, 0, 0) #'2010-01-04'

    start_date = datetime.datetime.combine(max_date, datetime.datetime.min.time()) + datetime.timedelta(days=1)
    end_date = datetime.datetime.now() - datetime.timedelta(days=1)

    date_now_timestamp = str(datetime.datetime.timestamp(start_date)).split('.')
    start = date_now_timestamp[0]

    date_now_timestamp = str(datetime.datetime.timestamp(end_date)).split('.')
    end = date_now_timestamp[0]

    symbol_data = get_data(symbol,start,end)

    if(symbol_data.empty):
        return '{"Result":"False"}'

    file_download = symbol_data.to_csv(header=True,index=False,encoding='utf-8-sig')        
    #name = 'YAHOO_FINANCE.csv'
    
    return file_download

In [4]:
yah = upload_azure()

In [5]:
yah_csv = StringIO(yah)
yah_pd = pd.read_csv(yah_csv, sep=',')

In [6]:
yah_pd.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,actualizacion
0,2010-01-05,1.0407,1.0410,1.0336,1.0395,1.0395,0.0,USDCAD=X,2020-10-18 20:38:42.780037
1,2010-01-06,1.0392,1.0407,1.0313,1.0319,1.0319,0.0,USDCAD=X,2020-10-18 20:38:42.780037
2,2010-01-07,1.0315,1.0369,1.0290,1.0340,1.0340,0.0,USDCAD=X,2020-10-18 20:38:42.780037
3,2010-01-08,1.0338,1.0382,1.0296,1.0298,1.0298,0.0,USDCAD=X,2020-10-18 20:38:42.780037
4,2010-01-11,1.0281,1.0341,1.0250,1.0338,1.0338,0.0,USDCAD=X,2020-10-18 20:38:42.780037


In [8]:
yah_pd.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,actualizacion
2803,2020-10-12,1.31310,1.31424,1.31009,1.31300,1.31300,0.0,USDCAD=X,2020-10-18 20:38:42.780037
2804,2020-10-13,1.31109,1.31456,1.30980,1.31125,1.31125,0.0,USDCAD=X,2020-10-18 20:38:42.780037
2805,2020-10-14,1.31396,1.31568,1.31164,1.31418,1.31418,0.0,USDCAD=X,2020-10-18 20:38:42.780037
2806,2020-10-15,1.31483,1.32560,1.31440,1.31486,1.31486,0.0,USDCAD=X,2020-10-18 20:38:42.780037
2807,2020-10-16,1.32284,1.32372,1.31772,1.32251,1.32251,0.0,USDCAD=X,2020-10-18 20:38:42.780037


In [7]:
yah_pd.to_csv('YAHOO_FINANCE.csv',index=False)

In [9]:
#Upload to Azure Blob Storage and Execute Manual Trigger.
import pyodbc
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

In [12]:
server = 'grainpredictive.database.windows.net'
database = 'gpi'
username = 'gpi'
password = 'Cmi@2019$A'   
driver= '{ODBC Driver 17 for SQL Server}'

In [13]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

In [14]:
query="""SELECT TOP (1000) [Symbol]
      ,MAX([Date])
      ,MAX([actualizacion])
  FROM [ST_YAHOO].[HISTORICAL_MARKET]
  GROUP BY [Symbol]"""

In [16]:
data = pd.read_sql(query, cnxn)
data

,Symbol,,
0,^DJI,2020-10-16,2020-10-19 08:02:26.513
1,^GSPC,2020-10-16,2020-10-19 08:02:26.587
2,ADM,2020-10-16,2020-10-19 08:02:26.697
3,BG,2020-10-16,2020-10-19 08:02:27.480
4,GC=F,2020-10-16,2020-10-19 08:02:27.580
5,GLEN.L,2020-10-16,2020-10-19 08:02:27.657
6,SI=F,2020-10-16,2020-10-19 08:02:27.760
7,USDBRL=X,2020-10-16,2020-10-19 08:02:27.820
8,USDCAD=X,2020-10-16,2020-10-18 20:38:42.780
9,USDRUB=X,2020-10-16,2020-10-19 08:02:27.920


In [ ]:
#Notes:
#https://finance.yahoo.com/quote/usdcad=x/